<a href="https://colab.research.google.com/github/ahanna13/MS_NN/blob/main/MS_NN_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is my first trial at a neural net for my dataset from RKI:

### Version 4.1 (20230306) of the MALDI-ToF Mass Spectrometry Database for Identification and Classification of Highly Pathogenic Microorganisms from the Robert Koch-Institute (RKI). It can be found at https://zenodo.org/records/7990990

In [ ]:
pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# ------ DATA PROCESSING ------

# This was what it gave me for a 2D input

In [ ]:
# Load the data
data = pd.read_csv('your_file.txt', sep=" ", header=None, names=['intensity'])

# Split the data into training and validation sets
train_df, val_df = train_test_split(data, test_size=0.2)

# Create image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='./',  # or your specific directory
    x_col='x',
    y_col='y',
    target_size=(64, 64),  # Resize images
    batch_size=32,
    class_mode='binary')  # or 'categorical' for multi-class

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='./',
    x_col='x',
    y_col='y',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary')

# This is what it gave me for a 1D input

In [ ]:
# Example loading and preprocessing
data = pd.read_csv('your_file.txt', sep=" ", header=None)
X = pad_sequences(data['x'].values)
y = to_categorical(data['y'].values)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ------ BUILD CNN MODEL ------

In [ ]:
model = Sequential([
    # input shape should be 66,000, 1
    Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1)),  # Adjust input shape based on your data; Ved said I may not need 32 since its spectral peaks
    MaxPooling1D(2),
    Conv1D(64, 3, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(128, activation='relu'), # Ved said to add another hidden layer
    # y_train.shape[1] should be 72 for the number of genera I have
    Dense(y_train.shape[1], activation='softmax')  # Assuming y is one-hot encoded
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ------ TRAIN MODEL ------

In [ ]:
# Ensure input data is correctly shaped
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32
)

# ------ EVALUATE MODEL ------

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

# I need to add https://wandb.ai/site